In [0]:
!pip3 install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow-addons

     |████████████████████████████████| 348.9MB 50kB/s 
     |████████████████████████████████| 501kB 8.8MB/s 
     |████████████████████████████████| 3.1MB 17.6MB/s 
     |████████████████████████████████| 1.0MB 2.7MB/s 


In [0]:
import tensorflow as tf
import os
import glob
import numpy as np
import numpy.random as npr
import pandas as pd
import tensorflow.keras as k
import tensorflow.keras.layers as l
import matplotlib.pyplot as plt
#import tensorflow_addons as tfa
import random
import cv2
import tensorboard
import datetime

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from keras import optimizers
from tensorboard import notebook
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
buffersize = 50
batchsize = 30
N_samples = 413
N_trainingsamples = 383
N_validationsamples = 30
N_prefetch = 8 
N_parallel_iteration = 4
N_testsamples = 103


"""Define 5 augmentation functions"""
@tf.function
def flip1(img):
   img_flipped = tf.image.random_flip_up_down(img)
   img = tf.cast(img_flipped, tf.float32) / 255.0
   return img

def flip2(img):
   img_flipped = tf.image.random_flip_left_right(img)
   img = tf.cast(img_flipped, tf.float32) / 255.0
   return img

    
def rotate(img):
    angles = tf.random.uniform([], minval=0, maxval=359, dtype=tf.dtypes.float32)
    img = tfa.image.rotate(img, angles, interpolation='NEAREST', name=None)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def zoom(img):
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 2))

    for i, scale in enumerate(scales):
        x1 = 0.5 - (0.5 * scale)
        y1 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1]
    # Create different crops for an image
    img = tf.image.crop_and_resize(img, boxes=boxes, box_indices=np.zeros(len(scales)), crop_size=(256, 256))
    # Create different crops for an image
    crops = tf.image.crop_and_resize([img], boxes=boxes, box_indices=np.zeros(len(scales)),
                                             crop_size=(256, 256))
    # Return a random crop
    return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]
            

def rot90(img):
    img = tf.image.rot90(img, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    img = tf.cast(img, tf.float32) / 255.0
    return img

In [0]:
"""build training and validation labels before spliting"""

csv_path = "/content/drive/My Drive/Masterlab/a. IDRiD_Disease Grading_Training Labels.csv"
files_csv = pd.read_csv(csv_path, usecols=[1])

"""The only difference of the input pipeline for transfer learning model is the reducion of the label dimmensions.
The labels are defined as numpy arrays with one axis."""

labels = np.zeros(shape=(N_samples,))  # save training and validation labels in one-hot coding form
csv_tensor = tf.convert_to_tensor(files_csv.values, dtype=tf.int32)
csv_tensor = tf.map_fn(lambda x: 1 if x > 1 else 0, files_csv.values)
for i in range(N_samples):
    if csv_tensor[i] == 1:
         labels[i][0] = 1
    else:
         labels[i][1] = 1


"""Read training and validation image files from folder"""
def load_file_names():
    files = glob.glob("/content/drive/My Drive/Masterlab/a. Training Set/*.jpg")
    return files


img_list = []
files = sorted(load_file_names())


"""The parse function converts a datapair(file, label) to a correspoding datapair(image, label)"""
def parse_train_function(files,labels):
    image_string = tf.io.read_file(files)
    image_decoded = tf.io.decode_jpeg(image_string)                # image is decoded
    image_resized = tf.image.resize_with_pad(image_decoded,        # image is normalized
                                             256,
                                             256)
 
    a = random.randint(1,10)              # chose a way to be augmented
    if a == 1:
      img = zoom(image_resized)
      img = tf.cast(img, tf.float32) / 255.0  
    elif a== 2:
      img = rotate(image_resized)
    elif a== 3:
      img = rot90(image_resized)
    elif a== 4:
      img = zoom(image_resized)    
    elif a== 5:
      img = flip1(image_resized)
    elif a== 6:
      img = flip2(image_resized)    
    else:
      img = image_resized  
      img = tf.cast(img, tf.float32) / 255.0       
    label = labels
    return img, label


"""validation datapair will be created as training datapair, but without data augmentation"""
def parse_val_function(files,labels): 
    image_string = tf.io.read_file(files)
    image_decoded = tf.io.decode_jpeg(image_string)
    image_resized = tf.image.resize_with_pad(image_decoded,
                                             256,
                                             256)
    img = tf.cast(image_resized, tf.float32) / 255.0
    label = labels
    return img, label

"""Build training dataset with its parse fuction"""
def build_train_ds(files, labels, batchsize):
    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    ds = ds.map(parse_train_function, N_parallel_iteration)
    ds = ds.shuffle(380).batch(batchsize).repeat(-1).prefetch(N_prefetch)
    return ds


"""Build validation dataset with its parse fuction"""
def build_val_ds(files, labels, batchsize):
    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    ds = ds.map(parse_val_function, N_parallel_iteration)
    ds = ds.shuffle(20).batch(30).prefetch(N_prefetch)
    return ds


"""Shuffle the index of file paths. It shortens the running time comparing to shuffling the a dataset with decoded images and labels"""
shuffle_idx = np.arange(0, N_samples)
np.random.shuffle(shuffle_idx)
files = [files[i] for i in shuffle_idx]
labels = [labels[i] for i in shuffle_idx]


"""Split the training and validation datasets by file path index"""
train_ds = build_train_ds(files[0:N_trainingsamples], labels[0:N_trainingsamples], batchsize)
val_ds = build_val_ds(files[N_trainingsamples:N_samples], labels[N_trainingsamples:N_samples], batchsize)


"""load the original images to check if file path are correct"""
img = plt.imread(files[100])
plt.imshow(img)
plt.title(labels[100])
plt.show()

In [0]:
"""Build testing dataset of images"""
N_testsamples = 103

def load_testfile_names():
    files = glob.glob("/content/drive/My Drive/Masterlab/b. Testing Set/*.jpg")
    return files


img_list_test = []
test_files = load_testfile_names()

for file in sorted(test_files):
    image_string = tf.io.read_file(file)
    image_decoded = tf.io.decode_image(image_string)
    image_resized = tf.image.resize_with_pad(image_decoded,   # Normalize image
                                             256,
                                             256)
    img = tf.cast(image_resized, tf.float32) / 255.0          # Normalize image
    img_list_test.append(img)
    
img_tensor_test = tf.convert_to_tensor(img_list_test, dtype=tf.float32)
img_test_ds = tf.data.Dataset.from_tensor_slices(img_tensor_test)


"""Load test label files"""
csv_path_test = "/content/drive/My Drive/Masterlab/b. IDRiD_Disease Grading_Testing Labels.csv"
files_csv_test = pd.read_csv(csv_path_test, usecols=[1])

"""The only difference of the input pipeline for transfer learning model is the reducion of the label dimmensions.
The labels are defined as numpy arrays with one axis."""

t = np.zeros(shape=(103,))      # Build test labels in one-hot coding form
csv_tensor = tf.convert_to_tensor(files_csv_test.values, dtype=tf.float32)
csv_tensor = tf.map_fn(lambda x: 1 if x > 1 else 0, csv_tensor)
for i in range(N_testsamples):
    if csv_tensor[i] == 1:
         t[i][0] = 1
    else:
         t[i][1] = 1


"""build testing dataset of labels"""
labels_ds_test = tf.data.Dataset.from_tensor_slices(t)
test_ds = tf.data.Dataset.zip((img_test_ds, labels_ds_test)).batch(batchsize)   # Combine the image dataset and label dataset for testset together

In [0]:
# build total dataset(testing set)
test_ds = tf.data.Dataset.zip((img_test_ds, labels_ds_test)).batch(batchsize)